# Training a model
see README.md for details

In [1]:
from tqdm import tqdm
import glob
from transformers import T5ForConditionalGeneration

# from transformers import    AutoTokenizer
# from transformers import TrainingArguments
# from transformers import Trainer
import torch

from config import data_root, model_root, checkpoint_name
from config import token_len, annot_len

import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)
torch.cuda.is_available()

True

In [4]:
num_special_tokens = 3
mname = "google/byt5-small"
model = T5ForConditionalGeneration.from_pretrained(mname)


In [5]:
input_list = []
label_list = []

for fname in tqdm(glob.glob(f"{data_root}/00-Prolog.train")):
# for fname in tqdm(glob.glob(f"{data_root}/*.train")):
# for fname in tqdm(glob.glob(f"{data_root}/06*.train")):
    # print(fname)
    with open(fname) as fin:
        for i, row in enumerate(fin):
            line = row[:-1].split("\t")
            input_list += [list(line[0].encode("utf-8"))]
            label_list += [list(line[1].encode("utf-8"))]
print(len(input_list))

100%|██████████| 1/1 [00:00<00:00, 222.43it/s]

784


In [6]:
input_len = token_len + annot_len
input_list = [l + [0] * (input_len - len(l)) for l in input_list]
input_ids = torch.tensor(input_list) + num_special_tokens

label_len = token_len
label_list = [l + [0] * (label_len - len(l)) for l in label_list]
labels = torch.tensor(label_list) + num_special_tokens


In [7]:
m = model(input_ids, labels=labels)
# m = model(input_list, labels=label_list)

OutOfMemoryError: CUDA out of memory. Tried to allocate 266.00 MiB. GPU 0 has a total capacty of 23.68 GiB of which 22.12 MiB is free. Process 3369637 has 23.65 GiB memory in use. Of the allocated memory 22.36 GiB is allocated by PyTorch, and 169.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.save_pretrained(f"{model_root}/{checkpoint_name}")

In [ ]:
del model
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
model